In [4]:

# run the test harness for evaluating a model
def run_test_harness():
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
    # evaluate model
    scores, histories = evaluate_model(trainX, trainY)
    # learning curves
    summarize_diagnostics(histories)
    # summarize estimated performance
    summarize_performance(scores)

# baseline cnn model for mnist
from numpy import mean
from numpy import std
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = mnist.load_data()
	# reshape dataset to have a single channel
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	# prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		# define model
		model = define_model()
		# select rows for train and test
		trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		# fit model
		history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=2)
		# evaluate model
		_, acc = model.evaluate(testX, testY, verbose=0)
		print('> %.3f' % (acc * 100.0))
		# stores scores
		scores.append(acc)
		histories.append(history)
	return scores, histories

# plot diagnostic learning curves
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# plot loss
		plt.subplot(2, 1, 1)
		plt.title('Cross Entropy Loss')
		plt.plot(histories[i].history['loss'], color='blue', label='train')
		plt.plot(histories[i].history['val_loss'], color='orange', label='test')
		# plot accuracy
		plt.subplot(2, 1, 2)
		plt.title('Classification Accuracy')
		plt.plot(histories[i].history['accuracy'], color='blue', label='train')
		plt.plot(histories[i].history['val_accuracy'], color='orange', label='test')
	plt.show()

# summarize model performance
def summarize_performance(scores):
	# print summary
	print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
	# box and whisker plots of results
	plt.boxplot(scores)
	plt.show()

# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# evaluate model
	scores, histories = evaluate_model(trainX, trainY)
	# learning curves
	summarize_diagnostics(histories)
	# summarize estimated performance
	summarize_performance(scores)

# entry point, run the test harness
run_test_harness()


Epoch 1/10
1500/1500 - 8s - loss: 0.1952 - accuracy: 0.9405 - val_loss: 0.0961 - val_accuracy: 0.9702 - 8s/epoch - 6ms/step
Epoch 2/10
1500/1500 - 8s - loss: 0.0641 - accuracy: 0.9813 - val_loss: 0.0638 - val_accuracy: 0.9805 - 8s/epoch - 5ms/step
Epoch 3/10


KeyboardInterrupt: 

In [5]:
import pandas as pd
from lcfunctions import load_lasair_lc, lasair_clean
# suppress warnings
import warnings
warnings.filterwarnings('ignore')

label_scheme = 'labels_2'

pd.options.display.max_columns = None

dataset = pd.read_csv(f'../processed_data/dataset_allfeatures_inc_labels.csv', low_memory=False)
dataset = dataset[dataset[label_scheme].notna()].reset_index(drop=True)
print(dataset[label_scheme].value_counts())
print(dataset.shape)

folderpath = '../lightcurves_dataset/lasair_2023_03_25'

# Create an empty 3D numpy array
length = 500
length_int = 1000
limit = 100
a = 10
bins = limit*a
max_n_obs = 0
len_vals_g = []
len_vals_r = []

allgdmdt = np.empty((0,))
allrdmdt = np.empty((0,))

X_g = np.empty((0, length)) # g band
X_g_int = np.empty((0, length_int)) # g band interpolated
X_gdmdt = np.empty((0, length)) # g band dm/dt
X_gjd = np.empty((0, length, 2)) # g band with julian dates
X_gdt = np.empty((0, length, 2)) # g band with time differences
X_gdmdt2 = np.empty((0, length, 2)) # g band with dm/dt and time differences
X_ghist = np.empty((0, bins)) # g band histogram

# As above but for r band
X_r = np.empty((0, length))
X_r_int = np.empty((0, length_int))
X_rdmdt = np.empty((0, length))
X_rjd = np.empty((0, length, 2))
X_rdt = np.empty((0, length, 2))
X_rdmdt2 = np.empty((0, length, 2))
X_rhist = np.empty((0, bins))

for obj in dataset['oid_ztf'].to_list():
    # Load and process lasair light curve
    lc_test = load_lasair_lc(oid=obj, path=folderpath)
    lc_appmag_test = lasair_clean(lc_test, limit=25, magerrlim=1)

    # Create a copy of the light curve
    df_lc = lc_appmag_test.copy()

    # Split the light curve into g and r bands
    df_lc_g = df_lc[df_lc['fid'] == 1].reset_index(drop=True)
    df_lc_r = df_lc[df_lc['fid'] == 2].reset_index(drop=True)

    # Get the mags and times
    lc_g = df_lc_g['dc_mag'].values
    lc_r = df_lc_r['dc_mag'].values
    try:
        lc_g_jd = (df_lc_g['jd'] - df_lc_g['jd'][0]).values
    except:
        lc_g_jd = []
    try:
        lc_r_jd = (df_lc_r['jd'] - df_lc_r['jd'][0]).values
    except:
        lc_r_jd = []


    # Interpolate the light curve to 1 day cadence or a set number of points
    try:
        lc_g_int = np.interp(np.arange(0, lc_g_jd[-1], 1), lc_g_jd, lc_g)
        lc_g_int = np.interp(np.arange(0,length_int,1), lc_g_jd, lc_g)
        # print(lc_g_int.shape)
    except:
        lc_g_int = np.array([])
    try:
        lc_r_int = np.interp(np.arange(0, lc_r_jd[-1], 1), lc_r_jd, lc_r)
        lc_r_int = np.interp(np.arange(0,length_int,1), lc_r_jd, lc_r)
    except:
        lc_r_int = np.array([])
    
    # Time differences between observations and prepend a zero to the array because the first observation has no time difference
    if len(lc_g_jd) > 0:
        lc_g_dt = np.diff(lc_g_jd)
        lc_g_dt = np.insert(lc_g_dt, 0, 0)
    else:
        lc_g_dt = np.array([])
    
    if len(lc_r_jd) > 0:
        lc_r_dt = np.diff(lc_r_jd)
        lc_r_dt = np.insert(lc_r_dt, 0, 0)
    else:
        lc_r_dt = np.array([])

    # Magnitude changes between observations and prepend a zero to the array because the first observation has no magnitude change
    if len(lc_g) > 0:
        lc_g_dm = np.diff(lc_g)
        lc_g_dm = np.insert(lc_g_dm, 0, 0)
    else:
        lc_g_dm = np.array([])
    if len(lc_r) > 0:
        lc_r_dm = np.diff(lc_r)
        lc_r_dm = np.insert(lc_r_dm, 0, 0)
    else:
        lc_r_dm = np.array([])

    # Get dm/dt and replace nan values with 0
    lc_g_dmdt = lc_g_dm / lc_g_dt
    lc_g_dmdt = np.nan_to_num(lc_g_dmdt, nan=0)
    lc_r_dmdt = lc_r_dm / lc_r_dt
    lc_r_dmdt = np.nan_to_num(lc_r_dmdt, nan=0)

    # Create a histogram of the magnitude derivatives
    lc_g_dmdt_cpy = lc_g_dmdt.copy()
    lc_r_dmdt_cpy = lc_r_dmdt.copy()
    # if value is greater than 50 or less than -50, set to 50 or -50 respectively
    lc_g_dmdt_cpy[lc_g_dmdt_cpy > limit] = limit
    lc_g_dmdt_cpy[lc_g_dmdt_cpy < -limit] = -limit
    lc_r_dmdt_cpy[lc_r_dmdt_cpy > limit] = limit
    lc_r_dmdt_cpy[lc_r_dmdt_cpy < -limit] = -limit
    lc_g_hist, lc_g_bins = np.histogram(lc_g_dmdt_cpy, bins=np.linspace(-limit, limit, bins+1))
    lc_r_hist, lc_r_bins = np.histogram(lc_r_dmdt_cpy, bins=np.linspace(-limit, limit, bins+1))
    # print(lc_g_bins, lc_r_bins.shape)
    # print(max(lc_g_hist))

    # Add the dm/dt to the array
    allgdmdt = np.append(allgdmdt, lc_g_dmdt, axis=0)
    allrdmdt = np.append(allrdmdt, lc_r_dmdt, axis=0)

    if len(lc_g) | len(lc_r) > max_n_obs:
        max_n_obs = max(len(lc_g), len(lc_r))
    
    len_vals_g.append(len(lc_g))
    len_vals_r.append(len(lc_r))

    # Make the array of length length, backfilling with zeros
    if len(lc_g) < length:
        lc_g = np.pad(lc_g, (length - len(lc_g), 0), 'constant', constant_values=(0, 0))
        lc_g_dm = np.pad(lc_g_dm, (length - len(lc_g_dm), 0), 'constant', constant_values=(0, 0))
        lc_g_dmdt = np.pad(lc_g_dmdt, (length - len(lc_g_dmdt), 0), 'constant', constant_values=(0, 0))
        lc_g_dt = np.pad(lc_g_dt, (length - len(lc_g_dt), 0), 'constant', constant_values=(0, 0))
        lc_g_jd = np.pad(lc_g_jd, (length - len(lc_g_jd), 0), 'constant', constant_values=(0, 0))
    elif len(lc_g) >= length:
        lc_g = lc_g[-length:]
        lc_g_dm = lc_g_dm[-length:]
        lc_g_dmdt = lc_g_dmdt[-length:]
        lc_g_dt = lc_g_dt[-length:]
        lc_g_jd = lc_g_jd[-length:]
    if len(lc_r) < length:
        lc_r = np.pad(lc_r, (length - len(lc_r), 0), 'constant', constant_values=(0, 0))
        lc_r_dm = np.pad(lc_r_dm, (length - len(lc_r_dm), 0), 'constant', constant_values=(0, 0))
        lc_r_dmdt = np.pad(lc_r_dmdt, (length - len(lc_r_dmdt), 0), 'constant', constant_values=(0, 0))
        lc_r_dt = np.pad(lc_r_dt, (length - len(lc_r_dt), 0), 'constant', constant_values=(0, 0))
        lc_r_jd = np.pad(lc_r_jd, (length - len(lc_r_jd), 0), 'constant', constant_values=(0, 0))
    elif len(lc_r) >= length:
        lc_r = lc_r[-length:]
        lc_r_dm = lc_r_dm[-length:]
        lc_r_dmdt = lc_r_dmdt[-length:]
        lc_r_dt = lc_r_dt[-length:]
        lc_r_jd = lc_r_jd[-length:]

    # Make the array of length length_int, backfilling with zeros
    if len(lc_g_int) < length_int:
        lc_g_int = np.pad(lc_g_int, (length_int - len(lc_g_int), 0), 'constant', constant_values=(0, 0))
    elif len(lc_g_int) >= length_int:
        lc_g_int = lc_g_int[-length_int:]
    if len(lc_r_int) < length_int:
        lc_r_int = np.pad(lc_r_int, (length_int - len(lc_r_int), 0), 'constant', constant_values=(0, 0))
    elif len(lc_r_int) >= length_int:
        lc_r_int = lc_r_int[-length_int:]

    
    # Add the light curve to the array
    X_g = np.append(X_g, [lc_g], axis=0)
    X_g_int = np.append(X_g_int, [lc_g_int], axis=0)
    X_gdmdt = np.append(X_gdmdt, [lc_g_dmdt], axis=0)
    X_gjd = np.append(X_gjd, [np.stack((lc_g, lc_g_jd), axis=1)], axis=0)
    X_gdt = np.append(X_gdt, [np.stack((lc_g, lc_g_dt), axis=1)], axis=0)
    X_gdmdt2 = np.append(X_gdmdt2, [np.stack((lc_g_dm, lc_g_dt), axis=1)], axis=0)
    X_ghist = np.append(X_ghist, [lc_g_hist], axis=0)
    
    X_r = np.append(X_r, [lc_r], axis=0)
    X_r_int = np.append(X_r_int, [lc_r_int], axis=0)
    X_rdmdt = np.append(X_rdmdt, [lc_r_dmdt], axis=0)
    X_rjd = np.append(X_rjd, [np.stack((lc_r, lc_r_jd), axis=1)], axis=0)
    X_rdt = np.append(X_rdt, [np.stack((lc_r, lc_r_dt), axis=1)], axis=0)
    X_rdmdt2 = np.append(X_rdmdt2, [np.stack((lc_r_dm, lc_r_dt), axis=1)], axis=0)
    X_rhist = np.append(X_rhist, [lc_r_hist], axis=0)




dwarf_nova_SU_UMa    630
dwarf_nova_Z_Cam     174
nova_like            144
nova_like_VY_Scl     120
dwarf_nova_U_Gem     116
polar                114
int_polar             49
AMCVn                 46
nova                  46
Name: labels_2, dtype: int64
(1439, 270)


In [31]:
# I want a multi-channel input with the following:
# 1. g band
# 2. r band
# Here is the new array using X_g and X_r
X_g_new = np.reshape(X_g, (X_g.shape[0], X_g.shape[1], 1))
X_r_new = np.reshape(X_r, (X_r.shape[0], X_r.shape[1], 1))
X_new = np.concatenate((X_g_new, X_r_new), axis=2)

# Now I want to a single channel but the colour. This would be X_g - X_r.
X_gmr = X_g - X_r
X_gmr_new = np.reshape(X_gmr, (X_gmr.shape[0], X_gmr.shape[1], 1))
X_gmr_new[0]

array([[ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.0

In [5]:
# cnn model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler

# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):

    version = '3'

    if version == '1':
        # load all train
        trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
        trainX = trainX[:,:,0:1]
        print(trainX.shape, trainy.shape)
        # load all test
        testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
        testX = testX[:,:,0:1]
        print(testX.shape, testy.shape)
        # zero-offset class values
        trainy = trainy - 1
        testy = testy - 1
        # one hot encode y
        trainy = to_categorical(trainy)
        testy = to_categorical(testy)
        print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	
    elif version == '2':
        # Create a dataset to test cnn on.
        X_variant1 = np.cos(np.arange(4, 9, 5e-3))
        X_variant1 = np.tile(X_variant1, (350, 1))
        X_variant1 = X_variant1 + np.random.normal(0, 0.9, X_variant1.shape)
        X_variant1a = np.sin(np.arange(0, 25, 0.025))
        X_variant1a = np.tile(X_variant1a, (150, 1))
        X_variant1a = X_variant1a + np.random.normal(0, 0.5, X_variant1a.shape)
        X_variant1 = np.concatenate((X_variant1, X_variant1a), axis=0)

        X_variant2 = np.cos(np.arange(4, 9, 5e-3))
        X_variant2 = np.tile(X_variant2, (500, 1))
        X_variant2 = X_variant2 + np.random.normal(0, 0.9, X_variant2.shape)

        # Create a third class of data that is of a different fuction
        X_variant3 = np.tan(np.arange(0, 20, 0.02))
        X_variant3 = np.tile(X_variant3, (500, 1))
        X_variant3 = X_variant3 + np.random.normal(0, 0.9, X_variant3.shape)

        X_variant = np.concatenate((X_variant1, X_variant2, X_variant3), axis=0)
        print(X_variant.shape)
        # When reshaping each of the values in the last dimension should be in an array
        X_variant = X_variant.reshape(-1, 1000, 1)
        y = np.zeros((X_variant.shape[0], 3))
        y[0:500, 0] = 1
        y[500:1000, 1] = 1
        y[1000:, 2] = 1

        # Train, test, split.
        trainX, testX, trainy, testy = train_test_split(X_variant, y, test_size=0.3, stratify=y, random_state=1)    
        print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	
    elif version == '3':
        X_variant = X_ghist
        X_variant = X_variant.reshape(-1, 1000, 1)
        # print(X_variant[0])

        # Get the labels as names
        y_names = dataset[label_scheme].to_numpy()
        # Encode the labels
        enc = LabelEncoder().fit(y_names)
        y_num = enc.transform(y_names)
        # One hot encode the labels
        y = to_categorical(y_num)

        # Train, test, split.
        trainX, testX, trainy, testy = train_test_split(X_variant, y, test_size=0.3, stratify=y, random_state=1)
        # Normalise the data
        mean = np.mean(trainX)
        std = np.std(trainX)
        trainX = (trainX - mean) / std
        testX = (testX - mean) / std

        print(trainX.shape, trainy.shape, testX.shape, testy.shape)

    return trainX, trainy, testX, testy

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 2, 150, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # model = Sequential()
    # model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    # model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    # model.add(Dropout(0.5))
    # model.add(MaxPooling1D(pool_size=2))
    # model.add(Flatten())
    # model.add(Dense(100, activation='relu'))
    # model.add(Dense(n_outputs, activation='softmax'))
    # model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network

    inputs = keras.Input(shape=(n_timesteps,n_features))
    # x = layers.BatchNormalization()(inputs)
    x = layers.Conv1D(filters=64, kernel_size=10, activation="ReLU", padding='valid')(inputs)
    # x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=64, kernel_size=10, activation="ReLU", padding='valid')(inputs)
    # x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=64, kernel_size=10, activation="ReLU", padding='valid')(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(100, activation="ReLU", kernel_regularizer=regularizers.l1_l2(l1=0.00, l2=0.00))(x)
    x = layers.Dense(100, activation="ReLU", kernel_regularizer=regularizers.l1_l2(l1=0.00, l2=0.00))(x)
    outputs = layers.Dense(9, activation="softmax")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  loss="categorical_crossentropy",
                  metrics=["categorical_accuracy"])
    
    # Balance the classes
    y_train_labels = np.argmax(trainy, axis=1)
    class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train_labels), y=y_train_labels)
    class_weights_dict = dict(zip(np.unique(y_train_labels), class_weights))

    model.fit(trainX, trainy, validation_data=(testX,testy),epochs=epochs, class_weight=class_weights_dict , batch_size=batch_size, verbose=verbose)
    
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    preds = model.predict(testX)
    cm = confusion_matrix(np.argmax(testy, axis=1), np.argmax(preds, axis=1))
    print(cm)
    return accuracy

# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(repeats=3):
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

# run the experiment
run_experiment()
# X_train, y_train, X_test, y_test = load_dataset()
# from matplotlib import pyplot as plt
# # print(X_train[20])
# set = X_train
# sety = y_train
# show = np.random.randint(0, set.shape[0])
# print(show)
# plt.plot(set[show], '.')
# plt.gca().invert_yaxis()
# print(sety[show])

(1007, 1000, 1) (1007, 9) (432, 1000, 1) (432, 9)
Epoch 1/150
32/32 - 1s - loss: 2.1865 - categorical_accuracy: 0.2175 - val_loss: 2.0121 - val_categorical_accuracy: 0.4144 - 1s/epoch - 33ms/step
Epoch 2/150
32/32 - 1s - loss: 2.0496 - categorical_accuracy: 0.2592 - val_loss: 1.7974 - val_categorical_accuracy: 0.4838 - 523ms/epoch - 16ms/step
Epoch 3/150
32/32 - 1s - loss: 1.9344 - categorical_accuracy: 0.2602 - val_loss: 1.8903 - val_categorical_accuracy: 0.1782 - 505ms/epoch - 16ms/step
Epoch 4/150
32/32 - 1s - loss: 1.8697 - categorical_accuracy: 0.2413 - val_loss: 1.7553 - val_categorical_accuracy: 0.3102 - 567ms/epoch - 18ms/step
Epoch 5/150
32/32 - 1s - loss: 1.7974 - categorical_accuracy: 0.2721 - val_loss: 1.7554 - val_categorical_accuracy: 0.3056 - 558ms/epoch - 17ms/step
Epoch 6/150
32/32 - 1s - loss: 1.7292 - categorical_accuracy: 0.3426 - val_loss: 1.8625 - val_categorical_accuracy: 0.2130 - 546ms/epoch - 17ms/step
Epoch 7/150
32/32 - 1s - loss: 1.6672 - categorical_accurac

[[  1   2   1   2   0   5   0   1   2]
 [  6 102   6  12   6  33   7   4  13]
 [  0   6  10   8   1   1   7   2   0]
 [  3   8   9  13   2   0   9   3   5]
 [  0   1   0   0   4   0   4   2   4]
 [  2   4   1   0   0   2   1   2   2]
 [  2   4   2   7   2   1  13   7   5]
 [  0   1   1   5   1   0  11  11   6]
 [  1   3   2   4   2   1   7   3  11]]
>#1: 38.657
Epoch 1/150
32/32 - 1s - loss: 2.1353 - categorical_accuracy: 0.1708 - val_loss: 1.9219 - val_categorical_accuracy: 0.3634 - 674ms/epoch - 21ms/step
Epoch 2/150
32/32 - 0s - loss: 2.0327 - categorical_accuracy: 0.2026 - val_loss: 1.9221 - val_categorical_accuracy: 0.1968 - 445ms/epoch - 14ms/step
Epoch 3/150
32/32 - 0s - loss: 1.9077 - categorical_accuracy: 0.1768 - val_loss: 1.7783 - val_categorical_accuracy: 0.3241 - 442ms/epoch - 14ms/step
Epoch 4/150
32/32 - 0s - loss: 1.8511 - categorical_accuracy: 0.2562 - val_loss: 1.7969 - val_categorical_accuracy: 0.2870 - 445ms/epoch - 14ms/step
Epoch 5/150
32/32 - 0s - loss: 1.7796 - 

[[  2   3   0   3   0   4   0   1   1]
 [ 10 112   7   9   2  29   5   7   8]
 [  0  18   6   5   0   0   3   2   1]
 [  2  11  12  11   3   0   4   5   4]
 [  0   1   0   2   2   1   3   3   3]
 [  2   6   1   0   0   1   1   1   2]
 [  3   8   4   5   3   2   7   8   3]
 [  0   4   3   4   4   1  10   6   4]
 [  3   7   2   3   1   0   5   5   8]]
>#2: 35.880
Epoch 1/150
32/32 - 1s - loss: 2.1785 - categorical_accuracy: 0.2582 - val_loss: 2.0807 - val_categorical_accuracy: 0.2477 - 691ms/epoch - 22ms/step
Epoch 2/150
32/32 - 0s - loss: 2.0183 - categorical_accuracy: 0.2989 - val_loss: 1.8227 - val_categorical_accuracy: 0.3611 - 441ms/epoch - 14ms/step
Epoch 3/150
32/32 - 0s - loss: 1.9059 - categorical_accuracy: 0.2910 - val_loss: 1.9221 - val_categorical_accuracy: 0.2292 - 447ms/epoch - 14ms/step
Epoch 4/150
32/32 - 0s - loss: 1.8787 - categorical_accuracy: 0.2830 - val_loss: 1.7490 - val_categorical_accuracy: 0.3032 - 436ms/epoch - 14ms/step
Epoch 5/150
32/32 - 0s - loss: 1.8033 - 